In [2]:
from utils import *
from pprint import pprint

In [3]:
house = importfile('hw3_house_votes_84.csv', ',')
housecategory = {}
for i in house[0]:
    housecategory[i] = 'categorical'
housecategory["class"] = 'class'
housedata = np.array(house[1:]).astype(float)

# print(len(housedata))
# print(housecategory)

#class in last column

In [4]:
wine = importfile('hw3_wine.csv', '\t')
winecategory = {}
for i in wine[0]:
    winecategory[i] = 'numerical'
winecategory["# class"] = 'class'
winedata = np.array(wine[1:]).astype(float)
# winedata = np.array([[float(item) for item in onelist] for onelist in wine[1:]])

# print(len(winedata))
# print(winecategory)

#class in first column

In [5]:
cancer = importfile('hw3_cancer.csv', '\t')
cancercategory = {}
for i in cancer[0]:
    cancercategory[i] = 'numerical'
cancercategory["Class"] = 'class'
cancerdata = np.array(cancer[1:]).astype(float)

#print(len(cancerdata))
#print(cancercategory)

#class in last column

In [6]:
cmc = importfile('cmc.data', ',')
cmccategory = {"Wife's age":"numerical","Wife's education":"categorical",
"Husband's education":"categorical","Number of children ever born":"numerical",
"Wife's religion":"binary","Wife's now working?":"binary",
"Husband's occupation":"categorical","Standard-of-living index":"categorical",
"Media exposure":"binary","Contraceptive method used":"class"}
cmcdata = np.array(cmc).astype(int)

# print(len(cmcdata))
# print(cmccategory)

#class in last column

In [26]:
w = {list(cmccategory.keys())[0]:cmccategory[list(cmccategory.keys())[0]]}

print(w)

16


In [ ]:
def id3bestseperate(dataset:np.array, attributes:dict):
    classindex = list(attributes.values()).index("class")
    originalentrophy = 0
    smallestentrophy = 1

    thresholdvalue = -1
    gain = 0.0
    bestattribution = {list(attributes.keys())[0]:attributes[list(attributes.keys())[0]]}
    for attribute in attributes:
        break
    # set first attribution dictionary {key:type} to the best attributes.
    return bestattribution, thresholdvalue, gain

In [ ]:
def id3cat(collist, listattribution):
    original_ent = entropy(collist[-1])
    smallest_ent = 1
    i = 0

    # bestindex = i
    best = listattribution[i]
    for attributes in listattribution[:-1]: # I keep the last column: the target/label.
        liskey = list(Counter(collist[i]).keys())
        listofcategory = []
        for value in liskey:
            index = [idx for idx, element in enumerate(collist[i]) if element == value]
            category = np.array(collist[-1][index]) 
            listofcategory.append(category) # list of nparrays of target/label/categories.

        ent = 0
        for cat in listofcategory:
            a = len(cat)/len(collist[i]) # This is probability
            ent += a * entropy(cat) # probability multiple by entropy

        if ent < smallest_ent:
            smallest_ent = ent
            best = attributes
            # bestindex = i
        i+=1

    return best, original_ent-ent
